Here is a good primer on calculating Uniswap v3 liquidity positions: https://blog.uniswap.org/uniswap-v3-math-primer-2

In [1]:
import json
import requests
import pandas as pd
import numpy as np

In [2]:
# @title query

query = """
query MyQuery {
  positions {
    account {
      id
    }
    id
    liquidity
    tickLower {
      index
    }
    tickUpper {
      index
    }
    pool {
      id
      tick
      inputTokens {
        name
        symbol
        id
        decimals
      }
    }
    timestampClosed
    timestampOpened
    hashOpened
    hashClosed
  }
}
"""

In [3]:
# pull raw Derpdex (Uniswap v3 fork) data from goldsky API and insert into pandas dataframe for analysis

url = "https://api.goldsky.com/api/public/project_clvq2euuzbinl01vgenyc84zw/subgraphs/deprdex3/1.0.0/gn"
response = requests.post(url, json={'query': query})
data = response.json()

positions = data['data']['positions']

df_list = []
for position in positions:
    input_tokens = position.get('pool', {}).get('inputTokens', [])
    
    # Ensure there are exactly two input tokens for each position
    if len(input_tokens) < 2:
        input_tokens.append({})
    
    df_list.append({
        'account_id': position.get('account', {}).get('id'),
        'position_id': position.get('id'),
        'liquidity': float(position.get('liquidity', 0)),
        'tickLower_index': position.get('tickLower', {}).get('index', '0'),
        'tickUpper_index': position.get('tickUpper', {}).get('index', '0'),
        'timestampClosed': position.get('timestampClosed'),
        'timestampOpened': position.get('timestampOpened'),
        'hashOpened': position.get('hashOpened'),
        'hashClosed': position.get('hashClosed'),
        'pool_id': position.get('pool', {}).get('id'),
        'pool_tick': position.get('pool', {}).get('tick'),
        'inputToken1_name': input_tokens[0].get('name'),
        'inputToken1_symbol': input_tokens[0].get('symbol'),
        'inputToken1_id': input_tokens[0].get('id'),
        'inputToken1_decimals': input_tokens[0].get('decimals'),
        'inputToken2_name': input_tokens[1].get('name'),
        'inputToken2_symbol': input_tokens[1].get('symbol'),
        'inputToken2_id': input_tokens[1].get('id'),
        'inputToken2_decimals': input_tokens[1].get('decimals')
    })
df = pd.DataFrame(df_list)

In [4]:
# show raw data from query

pd.set_option('display.float_format', '{:.10f}'.format)
df.head()

,account_id,position_id,liquidity,tickLower_index,tickUpper_index,timestampClosed,timestampOpened,hashOpened,hashClosed,pool_id,pool_tick,inputToken1_name,inputToken1_symbol,inputToken1_id,inputToken1_decimals,inputToken2_name,inputToken2_symbol,inputToken2_id,inputToken2_decimals
0,0xfaaadff49247bcf060a3d5ff1f0938625363df71,0x000000000000010000,0.0000000000,-887220,887220,1695204411,1694772737,0x86a380fdf543214b9d6dfb75f4b7843675dc3e31c4ea...,0xe8ed9095f263f9f8a638988182bab03b66e41eab0d25...,0x09b852108b6ed78adb2e776d901b8d54ba708080,-193785,Wrapped Ether,WETH,0x4200000000000000000000000000000000000006,18,USD Base Coin,USDbC,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,6
1,0x7d413382501807cdf7c5a99d687ff939fb8f1ef2,0x000000000000020000,944889177.0000000000,-887220,887220,None,1698229441,0x87ce8e04aa1e7b7adf982f6559bf2e3f023954c0ec2f...,None,0x09b852108b6ed78adb2e776d901b8d54ba708080,-193785,Wrapped Ether,WETH,0x4200000000000000000000000000000000000006,18,USD Base Coin,USDbC,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,6
2,0x440097cebd3b8c20ec43ba4a9395e252bf41dccc,0x000000000001000000,120064407092.0000000000,-887220,887220,None,1692304017,0xa15eb180d175143495cf2840a51e5c2e2716084e031e...,None,0x09b852108b6ed78adb2e776d901b8d54ba708080,-193785,Wrapped Ether,WETH,0x4200000000000000000000000000000000000006,18,USD Base Coin,USDbC,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,6
3,0xbba0f3816c020a48e28f9c9113b301d941776f05,0x000000000001010000,0.0000000000,-887220,887220,1695204411,1694772749,0x58867d836f22463c3e1548edc69626d7e61bfa876df5...,0x94c538cec436ffa57bfc1138d42d961be261d57142b4...,0x09b852108b6ed78adb2e776d901b8d54ba708080,-193785,Wrapped Ether,WETH,0x4200000000000000000000000000000000000006,18,USD Base Coin,USDbC,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,6
4,0x94f98fe46b3c96c77eacf297e86c8d47e5442ee7,0x000000000001020000,11853336055.0000000000,-887220,887220,None,1698231001,0xeb1a09ef157c785c527c60f546d26abdc83111ad65db...,None,0x09b852108b6ed78adb2e776d901b8d54ba708080,-193785,Wrapped Ether,WETH,0x4200000000000000000000000000000000000006,18,USD Base Coin,USDbC,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,6


We can verify the data is accurate by looking at the hashOppened:

1. Search the hash opened in block explorer
2. Take the tokenID generated by the hash, and use it to query the positions of the NonfungiblePositionManager contract

In [5]:
# Filter data

# Remove positions without DUSD (USDbC as example)
token_filter = '0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca'
df = df[(df['inputToken1_id'] == token_filter) | (df['inputToken2_id'] == token_filter)]

# If we want to filter out liquidity that is out of range / inactive

# df['is_active'] = df.apply(lambda row: row['timestampClosed'] is None and row['tickLower_index'] <= row['pool_tick'] <= row['tickUpper_index'], axis=1)
# df = df[df['is_active']]


In [6]:
# define functions for calculating user liquidity

def get_price_at_tick(tick):
  return 1.0001 ** tick

# Function to calculate the amount of token0
def calculate_amount0(liquidity, sqrt_price_current, sqrt_price_lower, sqrt_price_upper):
    if sqrt_price_current <= sqrt_price_lower:
        return liquidity * (sqrt_price_upper - sqrt_price_lower) / (sqrt_price_lower * sqrt_price_upper)
    elif sqrt_price_current < sqrt_price_upper:
        return liquidity * (sqrt_price_upper - sqrt_price_current) / (sqrt_price_current * sqrt_price_upper)
    else:
        return 0

# Function to calculate the amount of token1
def calculate_amount1(liquidity, sqrt_price_current, sqrt_price_lower, sqrt_price_upper):
    if sqrt_price_current <= sqrt_price_lower:
        return 0
    elif sqrt_price_current < sqrt_price_upper:
        return liquidity * (sqrt_price_current - sqrt_price_lower)
    else:
        return liquidity * (sqrt_price_upper - sqrt_price_lower)

In [7]:
# convert tick indices to integers
df['tickLower_index'] = df['tickLower_index'].astype(int)
df['tickUpper_index'] = df['tickUpper_index'].astype(int)
df['pool_tick'] = df['pool_tick'].astype(int)

# calculate sqrt prices (needed for calculating amounts)
df.loc[:, 'sqrt_price_lower'] = df['tickLower_index'].apply(lambda x: np.sqrt(get_price_at_tick(x)))
df.loc[:, 'sqrt_price_upper'] = df['tickUpper_index'].apply(lambda x: np.sqrt(get_price_at_tick(x)))
df.loc[:, 'sqrt_price_current'] = df['pool_tick'].apply(lambda x: np.sqrt(get_price_at_tick(x)))


# determine token0 and token1 based on their addresses (based on lexicographical order)
df.loc[:, 'token0'] = df.apply(lambda row: row['inputToken1_id'] if row['inputToken1_id'] < row['inputToken2_id'] else row['inputToken2_id'], axis=1)
df.loc[:, 'token1'] = df.apply(lambda row: row['inputToken1_id'] if row['inputToken1_id'] > row['inputToken2_id'] else row['inputToken2_id'], axis=1)

# determine the corresponding decimals and symbols for token0 and token1
df.loc[:, 'decimals0'] = df.apply(lambda row: row['inputToken1_decimals'] if row['token0'] == row['inputToken1_id'] else row['inputToken2_decimals'], axis=1)
df.loc[:, 'decimals1'] = df.apply(lambda row: row['inputToken1_decimals'] if row['token1'] == row['inputToken1_id'] else row['inputToken2_decimals'], axis=1)
df.loc[:, 'symbol0'] = df.apply(lambda row: row['inputToken1_symbol'] if row['token0'] == row['inputToken1_id'] else row['inputToken2_symbol'], axis=1)
df.loc[:, 'symbol1'] = df.apply(lambda row: row['inputToken1_symbol'] if row['token1'] == row['inputToken1_id'] else row['inputToken2_symbol'], axis=1)

# calculate raw amounts
df.loc[:, 'amount0'] = df.apply(lambda row: calculate_amount0(row['liquidity'], row['sqrt_price_current'], row['sqrt_price_lower'], row['sqrt_price_upper']), axis=1)
df.loc[:, 'amount1'] = df.apply(lambda row: calculate_amount1(row['liquidity'], row['sqrt_price_current'], row['sqrt_price_lower'], row['sqrt_price_upper']), axis=1)

# convert to actual token amounts by applying decimals
df.loc[:, 'amount0_actual'] = df['amount0'] / (10 ** df['decimals0'])
df.loc[:, 'amount1_actual'] = df['amount1'] / (10 ** df['decimals1'])


In [9]:
# Now we have the final amounts, with the actual USDbC liqudity which can be utilized to calculate the user's points
df[['account_id','hashOpened','amount0_actual', 'symbol0', 'amount1_actual', 'symbol1']]

,account_id,hashOpened,amount0_actual,symbol0,amount1_actual,symbol1
0,0xfaaadff49247bcf060a3d5ff1f0938625363df71,0x86a380fdf543214b9d6dfb75f4b7843675dc3e31c4ea...,0.0000000000,WETH,0.0000000000,USDbC
1,0x7d413382501807cdf7c5a99d687ff939fb8f1ef2,0x87ce8e04aa1e7b7adf982f6559bf2e3f023954c0ec2f...,0.0000152461,WETH,0.0585603213,USDbC
2,0x440097cebd3b8c20ec43ba4a9395e252bf41dccc,0xa15eb180d175143495cf2840a51e5c2e2716084e031e...,0.0019372769,WETH,7.4410951363,USDbC
3,0xbba0f3816c020a48e28f9c9113b301d941776f05,0x58867d836f22463c3e1548edc69626d7e61bfa876df5...,0.0000000000,WETH,0.0000000000,USDbC
4,0x94f98fe46b3c96c77eacf297e86c8d47e5442ee7,0xeb1a09ef157c785c527c60f546d26abdc83111ad65db...,0.0001912573,WETH,0.7346207207,USDbC
...,...,...,...,...,...,...
94,0xb55e9c792fca2c76cb37cc396df7652959790c86,0x2e64a9b58bb0096da526a3aed6c9a6ab4a5c20378ff7...,0.0000067401,WETH,0.0258889080,USDbC
95,0x0a251df99a88a20a93876205fb7f5faf2e85a481,0x8b30989e78339d1953bd7b6679cad86f040b996dfa68...,0.0000000000,WETH,0.0000000000,USDbC
96,0x58fd7ea0ac6af968936fb7863bf8c68835297cd9,0x9c2cf35771876429dd97c8105ea6cb853fabb4f02fd3...,0.0008037304,WETH,3.0871346017,USDbC
97,0x6448f86bc19d217007c945632159f148fd6490a6,0xa93eaf99d5f0e2fb7ffb7ba945a32b31e3fe0518b302...,0.0000019928,WETH,0.0076544309,USDbC
